## Importing Libraries

In [69]:
import pandas as pd
import numpy as np
import time
from selenium import webdriver
from selenium.webdriver.common.by import By  

## Connecting selenium to Chrome

In [70]:
driver=webdriver.Chrome()

## Opening Website 

In [71]:
driver.get("https://www.gyanboard.com/")

## Loop that Scrap Pages Content line by Line

In [6]:
table_=[]
for i in range(100):
    try:
        content=driver.find_elements(By.XPATH,'//section')
        content_=content[1].text
#         content_=content_.replace("\n"," ").replace("  "," ").replace("*","")
        content_.split(" Next >>")[0].split(" << Prev")[0]
        table_.append([content[0].text.split("\n")[0].strip(),content_.strip()])
        time.sleep(2)
        driver.find_element(By.XPATH,'//button[@style="float:inline-end"]').click()
    except NoSuchElementException:
        print("No more pages. Scraping complete.")
        break

In [25]:
driver.find_element(By.XPATH,'//button[@class="btn btn-primary"]').click()

## Managing Scraped Data to a Tabular Data (Table)

In [91]:
df2=pd.DataFrame(table_)

In [89]:
df=pd.concat([df1,df2])

In [101]:
df[1]=df[1].apply(lambda x: x.strip())

## Scarping the Topic Names of Chapters

In [176]:
chapter_name = driver.find_elements(By.XPATH,'//li[@class="nav-heading"]')
chapters_data={}

In [197]:
chapters_data=[]
chapters = driver.find_elements(By.XPATH, '//li[@class="nav-heading"]')
for i in range(len(chapters)):
    sibling = chapters[i].find_elements(By.XPATH, 'following-sibling::*')
    j=0
    temp_=[]
    if sibling:
        while sibling[j].text != chapters[i+1].text:
            temp_.append(sibling[j].text.strip())
            j+=1
        chapters_data.append([chapters[i].text.strip(), temp_])
   

In [198]:
chapter_data = pd.DataFrame(chapters_data)

In [199]:
chapter_data = chapter_data.explode(1)

## Merginng those Topic names to original Table

In [201]:
new_df=pd.merge(df,chapter_data,left_on=0,right_on=1,how="left")

In [203]:
new_df.drop(columns=['key_0','0_x'],inplace=True)

In [206]:
new_df.rename(columns={
    '0_y':'Chapter',
    '1_y':'Topic',
     '1_x':'Content'
},inplace=True)

In [208]:
new_df = new_df[['Chapter','Topic','Content']]

## Final Table

In [209]:
new_df

,Chapter,Topic,Content
0,Chapter 1 - Matter in Our Surroundings,States of matter,Matter around us exists in their different sta...
1,Chapter 1 - Matter in Our Surroundings,Can matter change its state?,Effect of change of temperature:- On increasin...
2,Chapter 1 - Matter in Our Surroundings,Physical nature of matter and Characteristics ...,Anything that occupies space and has mass and ...
3,Chapter 1 - Matter in Our Surroundings,Evaporation,Evaporation: Change of a liquid into vapours a...
4,Chapter 2- Is matter around us pure,Introduction of Is Matter Around Us Pure,Anything that occupies space and has mass is c...
...,...,...,...
78,Chapter 12 - Sound,Characteristics Of Sound Wave,We can describe a sound wave by its frequency ...
79,Chapter 13 - Why Do We Fall ill,Disease and Its Causes,What does disease look like? :- There are many...
80,Chapter 13 - Why Do We Fall ill,Differences between Being Healthy and Disease-...,Being Healthy Being Disease-free It is a state...
81,Chapter 13 - Why Do We Fall ill,Health and its Failure,The significance of Health:- Health is a state...


## Exporting it in CSV format

In [210]:
new_df.to_csv('Gyan_Board.csv')